## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-25-21-50-22 +0000,nyt,Justice Dept. Official Pushes Prosecutors to I...,https://www.nytimes.com/2025/09/25/us/politics...
1,2025-09-25-21-47-59 +0000,nyt,911 Service Is Disrupted Across Louisiana and ...,https://www.nytimes.com/2025/09/25/us/mississi...
2,2025-09-25-21-47-40 +0000,nyt,Abbas’s Message on Gaza to the United Nations,https://www.nytimes.com/2025/09/25/briefing/ab...
3,2025-09-25-21-47-03 +0000,nypost,Judges rules in favor of pot shops in dispute ...,https://nypost.com/2025/09/25/us-news/judges-r...
4,2025-09-25-21-47-00 +0000,wsj,Opinion | Charlie Kirk and the New Christian GOP,https://www.wsj.com/opinion/charlie-kirk-and-t...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
125,trump,50
34,ice,18
260,president,16
33,dallas,14
90,government,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
179,2025-09-25-14-00-00 +0000,wsj,The Trump administration will offer AI models ...,https://www.wsj.com/tech/ai/trump-elon-musk-xa...,127
36,2025-09-25-21-02-00 +0000,wsj,Fed governor Lisa Cook urged the Supreme Court...,https://www.wsj.com/politics/policy/fed-govern...,118
73,2025-09-25-19-27-11 +0000,nypost,White House reposts Tylenol’s own old tweet wa...,https://nypost.com/2025/09/25/us-news/white-ho...,107
139,2025-09-25-16-25-34 +0000,nypost,Disney gears up for court battle with Trump af...,https://nypost.com/2025/09/25/business/disney-...,103
111,2025-09-25-17-49-10 +0000,nypost,Trump calls out Turkish president for buying R...,https://nypost.com/2025/09/25/us-news/trump-ca...,100


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
179,127,2025-09-25-14-00-00 +0000,wsj,The Trump administration will offer AI models ...,https://www.wsj.com/tech/ai/trump-elon-musk-xa...
83,80,2025-09-25-19-05-21 +0000,nypost,Anti-ICE gunman Joshua Jahn had ‘hatred’ for g...,https://nypost.com/2025/09/25/us-news/anti-ice...
36,68,2025-09-25-21-02-00 +0000,wsj,Fed governor Lisa Cook urged the Supreme Court...,https://www.wsj.com/politics/policy/fed-govern...
141,56,2025-09-25-16-17-15 +0000,nypost,Over 100 ex-ABC journalists urge Disney’s Bob ...,https://nypost.com/2025/09/25/business/over-10...
49,52,2025-09-25-20-27-00 +0000,wsj,The Bank of Mexico cut its benchmark interest ...,https://www.wsj.com/articles/bank-of-mexico-ma...
317,50,2025-09-24-23-28-04 +0000,nyt,Dallas ICE Facility Shooting Leaves One Detain...,https://www.nytimes.com/2025/09/24/us/dallas-i...
129,44,2025-09-25-16-58-18 +0000,cbc,Liberal government frees Canada Post to end ho...,https://www.cbc.ca/news/politics/canada-post-t...
122,39,2025-09-25-17-10-56 +0000,nypost,Chinese drone experts have been working with R...,https://nypost.com/2025/09/25/world-news/chine...
142,38,2025-09-25-16-16-13 +0000,nypost,"Starbucks slashing 900 jobs, closing hundreds ...",https://nypost.com/2025/09/25/business/starbuc...
246,37,2025-09-25-09-32-15 +0000,nypost,Former French President Sarkozy found guilty o...,https://nypost.com/2025/09/25/world-news/forme...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
